# Motivation

The purpose of this notebook is to introduce a simpler method but with the same results or similar to extract the 98 TFs

In [1]:
%load_ext autoreload
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import os
import sys

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import multiprocess as mp


# own libraries
sys.path.append('/Users/vlad/Developer/York/iNet_v2/src/')

from NetworkAnalysis.ExperimentSet import ExperimentSet
from NetworkAnalysis.NetworkOutput import NetworkOutput

from NetworkAnalysis import GraphHelper as gh
from NetworkAnalysis.utilities import clustering as cs
from NetworkAnalysis.utilities import sankey_consensus_plot as sky
from NetworkAnalysis.utilities.helpers import save_fig, survival_plot
from NetworkAnalysis.GraphToolExp import GraphToolExperiment as GtExp
from NetworkAnalysis.dea import dea
from NetworkAnalysis.dea import helpers as dea_hp
from NetworkAnalysis.dea import gsea as gsea_hp
# Gsea libraries

pio.templates.default = "ggplot2"

pool = mp.Pool(mp.cpu_count())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
results_path = "../../results/exp/"
data_base = "../../data/"
base_path = "../../results/"


# Experiments set
exp_folder_tumour_42 = "network_I/gc_42/tum/" 
exp_folder_tumour_47 = "network_I/gc_47/tum/" 

exp_folder_h42_ctrl = '../results/network_I/gc_42/tum_ctrls/'


figures_path = "./tum_sel_pruning/gc_42/"

vu_net_1 = pd.read_csv(f"{data_base}/net_I/vu_net_1.tsv", sep="\t", index_col="Sample")
vu_output = pd.read_csv(f"{data_base}/metadata/VU_clustering_v3.tsv", sep="\t", index_col="Sample")

# Net I labels
vu_output["Net_I"] = vu_net_1["dendrogram_label"]
vu_output["Net_I"] = vu_output["Net_I"].fillna("Na")

tcga_mutations_df = pd.read_csv(f"{data_base}/tumour/mutations_tcga.csv")
tcga_mutations_df = tcga_mutations_df[tcga_mutations_df["count"] != 0].set_index("gene")

tum_tpms = pd.read_csv(f"{data_base}/tumour/TPMs_selected_genes_v3_13k_gc42.tsv", sep="\t", index_col="genes")
tum_tpms_v4 = pd.read_csv(f"{data_base}/tumour/tum_TPMs_selected_genes_gc42_all_v4.tsv", sep="\t", index_col="genes")

# Healthy and metadata
# healthy_metadata = pd.read_csv(f"{data_base}/non_cancerous/healthy_bladder_metadata.tsv", sep="\t", index_col="Sample").drop(index=["Y2796_P0"])
# healthy_metadata["sample_prcsd"] = healthy_metadata.index.str.replace("-", "_")

healthy_tpm = pd.read_csv(f"{data_base}/non_cancerous/healthy_data_all_gc42_v4.tsv", sep="\t", index_col="gene")
remap_col = {}
for col in healthy_tpm.columns:
    remap_col[col] = col.replace("-", "_")
healthy_tpm = healthy_tpm.rename(columns=remap_col)

# tf list
tf_path = f"{data_base}/metadata/TF_names_v_1.01.txt"
if os.path.exists(tf_path):
    tf_list = np.genfromtxt(fname=tf_path, delimiter="\t", skip_header=1, dtype="str")

In [3]:
# Base-line, this is the results from previous work
h_sel_tfs = pd.read_csv(f"{data_base}/tf_ctrl_v2.tsv", index_col="gene", sep="\t")
h_sel_tfs["tum_mean_expression"] = tum_tpms_v4.loc[h_sel_tfs.index].mean(axis=1)
h_sel_tfs["tum_median_expression"] = tum_tpms_v4.loc[h_sel_tfs.index].median(axis=1)
h_sel_tfs["tum_std_expression"] = tum_tpms_v4.loc[h_sel_tfs.index].std(axis=1)
h_sel_tfs["tum_var_expression"] = tum_tpms_v4.loc[h_sel_tfs.index].var(axis=1)

# Tum
t_sel_tfs = pd.read_csv(f"{data_base}/gc42_tum_tf_ctrl_1.tsv", index_col="gene", sep="\t")
t_sel_tfs["tum_mean_expression"] = tum_tpms_v4.loc[t_sel_tfs.index].mean(axis=1)
t_sel_tfs["tum_median_expression"] = tum_tpms_v4.loc[t_sel_tfs.index].median(axis=1)
t_sel_tfs["tum_std_expression"] = tum_tpms_v4.loc[t_sel_tfs.index].std(axis=1)
t_sel_tfs["tum_var_expression"] = tum_tpms_v4.loc[t_sel_tfs.index].var(axis=1)

In [31]:
len(list(set(h_sel_tfs.index) & set(t_sel_tfs.index)))

23

In [41]:
t_sel_tfs

,mut_count,tum_median_expression,healthy_median_expression,tum_mean_expression,healthy_mean_expression,tum_std_expression,healthy_std_expression,tum_var_expression
gene,,,,,,,,
ZNF43,6.0,5.251136,6.146340,6.736914,7.766050,5.574155,6.505078,31.071206
ZNF91,20.0,10.695356,12.547543,13.280044,15.979586,10.193745,11.890302,103.912436
ZNF841,7.0,4.307739,4.212624,5.990788,6.004328,6.832527,6.491802,46.683425
KLF11,4.0,6.289583,5.911865,7.778770,7.292082,5.976297,5.616870,35.716130
CREB3L2,5.0,16.027827,15.461771,18.843924,18.041244,13.431502,12.801455,180.405251
...,...,...,...,...,...,...,...,...
ZNF827,10.0,4.258290,4.370418,4.988981,5.256739,3.366387,3.814307,11.332559
POU2AF1,3.0,3.002048,2.790213,7.282661,6.828753,20.873332,19.608442,435.695976
KLF9,0.0,5.029436,4.671094,7.123853,6.682199,7.617043,7.159530,58.019349


## Difference in the TF list

In [4]:
exp_folder_tumour_42 = "network_I/gc_42/tum/" 
exp_folder_tumour_47 = "network_I/gc_47/tum/" 

In [35]:
%autoreload 2
tum_42 = ExperimentSet("tum", base_path, exp_folder_tumour_42, tcga_mutations_df, sel_sets = ["5K",], rel_path="../", exp_type='iNet')
sel_exp_42 = tum_42.exps['standard_5K_6TF_hsbm']

tum_47 = ExperimentSet("tum", base_path, exp_folder_tumour_47, tcga_mutations_df, sel_sets = ["5K",], rel_path="../", exp_type='iNet')
sel_exp_47 = tum_47.exps['standard_5K_6TF_hsbm']

exp_folder_h42 = "network_I/healthy42/"
all_h = ExperimentSet("allHealthy42", base_path, exp_folder_h42, tcga_mutations_df, sel_sets = ["5K"], rel_path="../")
sel_exp_h = all_h.exps['standard_5K_6TF']


##### Experiment labels:  dict_keys(['standard_5K_12TF_hsbm', 'standard_5K_11TF_hsbm', 'standard_5K_4TF_hsbm', 'standard_5K_9TF_hsbm', 'standard_5K_10TF_hsbm', 'standard_5K_5TF_hsbm', 'standard_5K_3TF_hsbm', 'standard_5K_8TF_hsbm', 'standard_5K_7TF_hsbm', 'standard_5K_6TF_hsbm'])
##### Experiment labels:  dict_keys(['standard_5K_8TF_hsbm', 'standard_5K_3TF_hsbm', 'standard_5K_11TF_hsbm', 'standard_5K_9TF_hsbm', 'standard_5K_6TF_hsbm', 'standard_5K_10TF_hsbm', 'standard_5K_5TF_hsbm', 'standard_5K_4TF_hsbm', 'standard_5K_7TF_hsbm', 'standard_5K_12TF_hsbm'])
##### Experiment labels:  dict_keys(['standard_5K_3TF', 'norm3_5K_3TF', 'beta_5K_3TF', 'standard_5K_6TF', 'standard_5K_10TF', 'norm3_5K_6TF', 'beta_5K_6TF', 'norm3_5K_10TF', 'beta_5K_10TF', 'standard_5K_4TF', 'standard_5K_7TF', 'standard_5K_8TF', 'standard_5K_5TF', 'standard_5K_9TF', 'norm3_5K_4TF', 'beta_5K_4TF', 'standard_5K_15TF', 'standard_5K_20TF', 'standard_5K_30TF', 'standard_5K_25TF', 'standard_5K_40TF', 'standard_5K_35TF', 's

In [6]:
cmn_42 = set(sel_exp_42.tpm_df.index) & set(h_sel_tfs.index)
print(f"GCv42 & 98 TFs. Common genes {len(cmn_42)}")

cmn_47 = set(sel_exp_47.tpm_df.index) & set(h_sel_tfs.index)
print(f"GCv47 & 98 TFs. Common genes {len(cmn_47)}")

cmn_47_42 = set(sel_exp_47.tpm_df.index) & set(sel_exp_42.tpm_df.index)
print(f"GCv47 & GCv42. Common genes {len(cmn_47_42)}")

cmn_47_42 = set(sel_exp_47.tpm_df.index) & set(sel_exp_42.tpm_df.index) & set(h_sel_tfs.index)
print(f"GCv47 & GCv42 & 98 TFs. Common genes {len(cmn_47_42)}")

GCv42 & 98 TFs. Common genes 44
GCv47 & 98 TFs. Common genes 43
GCv47 & GCv42. Common genes 3842
GCv47 & GCv42 & 98 TFs. Common genes 43


In [42]:
tf_inc_t_42 = set(sel_exp_42.tpm_df.index) & set(tf_list)
print(f"Tum GCv42 & TFs. Common genes {len(tf_inc_t_42)}")

tf_inc_h_42 = set(sel_exp_h.tpm_df.index) & set(tf_list)
print(f"Healthy GCv42 & TFs. Common genes {len(tf_inc_h_42)}")

tf_inc_h_t_42 = set(sel_exp_h.tpm_df.index) & set(tf_list) & set(sel_exp_42.tpm_df.index)
print(f"Healthy & tumour GCv42 & TFs. Common genes {len(tf_inc_h_t_42)}")

tf_inc_t_47 = set(sel_exp_47.tpm_df.index) & set(tf_list)
print(f"Tum GCv47 & TFs. Common genes {len(tf_inc_t_47)}")


Tum GCv42 & TFs. Common genes 359
Healthy GCv42 & TFs. Common genes 325
Healthy & tumour GCv42 & TFs. Common genes 156
Tum GCv47 & TFs. Common genes 324


In [49]:
pd.Series(list(tf_inc_t_42), name='gene').to_csv(f'{data_base}/gc42_tum_tfs_incl.tsv', sep='\t')
pd.Series(list(tf_inc_h_42), name='gene').to_csv(f'{data_base}/gc42_h_tfs_incl.tsv', sep='\t')

## Import the data

## Experiment sets

In [7]:
def load_experiment_set(base_path, exp_folder_path):
    exp_path = f"{base_path}/{exp_folder_path}/Stats/"
    meta_files = next(os.walk(exp_path), (None, None, []))[2]
    # keep only the network stats
    files = [file for file in meta_files if "networkStats" in file]
    exp_set = {}
    for file in files:
        exp = file.replace(".tsv", "").replace("networkStats_tum_standard_", "")
        exp_set[exp] = pd.read_csv(f"{exp_path}/{file}", index_col="gene", sep="\t")
    return exp_set

In [8]:
folders = next(os.walk(base_path + exp_folder_h42_ctrl), (None, None, []))[1]
exp_ctrls = {}
for folder in folders:
    hCtrl_path = f"{exp_folder_h42_ctrl}/{folder}/"
    meta_files = next(os.walk(hCtrl_path), (None, None, []))[2]
    # keep only the network stats
    files = [file for file in meta_files if "networkStats" in file]
    exp_ctrls[folder] = load_experiment_set(base_path, hCtrl_path)

exp_set = load_experiment_set(base_path, exp_folder_tumour_42)

## Network stats

Information calculated in the selective edge pruning

In [9]:
def merge_metrics(exp_set: dict, exp_type: str):
    comb_df = pd.DataFrame()
    for key, metrics in exp_set.items():
        metrics["TF"] = int(key.split("_")[-1].replace("TF", ""))
        metrics["Type"] = exp_type
        comb_df = pd.concat([comb_df, metrics], axis=0)

    return comb_df


# Experiments
all_exps = merge_metrics(exp_set=exp_set, exp_type="Exp")
all_exps["isCtrl"] = False

# Ctrls
all_ctrls = pd.DataFrame()
for key, value in exp_ctrls.items():
    label = key.split("h")[-1]
    ctrl_exps = merge_metrics(exp_set=value, exp_type=key)
    ctrl_exps["isCtrl"] = True
    all_ctrls = pd.concat([all_ctrls, ctrl_exps], axis=0)

# Combined
comb_exps = pd.concat([all_exps, all_ctrls], axis=0)


sel_tfs_metrics = comb_exps.loc[comb_exps.index.isin(t_sel_tfs.index)].copy(deep=True)
# sel_tfs_metrics = comb_exps

reorder_cols = [
    "degree_w",
    "degree_t",
    "degree_i",
    "degree_o",
    "pageRank",
    "closeness",
    "katz",
    "hits",
    "central_point_dominance",
    "max_b",
    "TF",
    "betweenness",
    "marked_as_tf",
    "mut_count",
    "Type",
    "isCtrl",
    "ModCon_Rank",
]
sel_tfs_metrics = sel_tfs_metrics[reorder_cols]
comb_exps = comb_exps[reorder_cols]

# Correlations between the metrics

In [10]:
import seaborn as sns
import matplotlib.pyplot as plt

if 0:
    corr_df = sel_tfs_metrics.drop(
        columns=[
            "Type",
            "isCtrl",
            "degree_i",
            "marked_as_tf",
            "max_b",
        ]
    ).corr(method="spearman")

    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_df, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
    plt.title("Correlation Heatmap")

    plt.savefig(f"{figures_path}/sel_tfs_corr_heatmap.png")
    plt.show()

# Matching with the 98 TFs

## Experiment networks

In [11]:
# We want to sort by the order of TF, but first we have to extract the TF values
def sort_exp_set(exp_set: dict):
    sorted_tf = {}
    for key in exp_set.keys():
        prscsd_label = int(key.split("_")[-1].replace("TF", ""))
        sorted_tf[prscsd_label] = key
    sorted_tf = dict(sorted(sorted_tf.items()))

    # Now we can sort the experiment set
    sorted_exp = {}
    for key in sorted_tf.values():
        sorted_exp[key] = exp_set[key]

    return sorted_exp

def calculate_degree_thresholds(exp_set: dict, sel_tfs: pd.DataFrame, label: str, q_th=0.75, verbose=False):

    sel_tfs_set = set()
    if not sel_tfs.empty:
        sel_tfs_set = set(sel_tfs.index)

    top_genes = set()  # genes that are the top set quartile th
    meta_info = []  # meta info about the TF experiments, the genes matched and the evolution across the experiments

    for exp, metrics in exp_set.items():
        deg_th = metrics["degree_t"].quantile(q=q_th)
        genes = set(metrics[metrics["degree_t"] >= deg_th].index)
        tf_exp = int(exp.split("_")[-1].replace("TF", ""))

        # Genes matched, and uniqued to this TF experiment
        single_exp_matched = set(genes) & sel_tfs_set
        unique = single_exp_matched - (top_genes & sel_tfs_set)

        # Add the new genes
        top_genes.update(genes)

        meta_info.append((tf_exp, deg_th, len(genes), len(unique), len(single_exp_matched), list(genes), list(unique), single_exp_matched))

    if sel_tfs.empty:
        matched = top_genes
    else:
        matched = top_genes & set(sel_tfs.index)

    if verbose:
        print(f"--> {label}\n\tQ_th: {q_th}. Matched genes with 98 TFs {len(matched)}. Selected {len(top_genes)}")

    # Combine in a DataFrame
    dmy = pd.DataFrame(meta_info, columns=["TF", "Degree_th", "selected", "unique", "matched", "selected genes", "unique genes", "matched genes"])
    dmy = dmy.sort_values(by=["TF"], ascending=True)
    dmy["TF"] = dmy["TF"].astype(str)
    # If experiment or control
    dmy["type"] = label

    # global stats
    global_stats = {"matched": len(matched), "selected": len(top_genes), "matched_genes": list(matched), "selected_genes": list(top_genes)}
    fig = px.bar(dmy, x="TF", y=["matched", "unique", "selected"], text_auto=True, barmode="group", title=f"Q_th: {q_th}. For {label}")

    # Unselect the 'selected' trace in the plotly figure
    fig.for_each_trace(lambda trace: trace.update(visible="legendonly") if trace.name == "selected" else ())

    return matched, fig, dmy, global_stats

In [12]:
q_ths = list(range(50, 100, 5))
q_ths.sort()
stats = [(0, len(t_sel_tfs))]  # default values
exp_results_qth = pd.DataFrame()
for q_th in q_ths:
    q_th = q_th / 100
    matched_exp, fig_exp, meta_info, global_stats = calculate_degree_thresholds(exp_set=exp_set, sel_tfs=t_sel_tfs, label="Exp", q_th=q_th, verbose=False)
    meta_info["Quantile"] = q_th
    exp_results_qth = pd.concat([exp_results_qth, meta_info], axis=0)

In [13]:
fig = px.line(
    exp_results_qth,
    x="TF",
    y="matched",
    color="Quantile",
    markers=True,
    color_discrete_sequence=px.colors.qualitative.G10,
    title="Inclusion of the 98 TFs genes",
)
fig = fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        xanchor="center",
        x=0.6,
        y=0.02,
        bgcolor="rgba(0,0,0,0)",
        font=dict(size=20, color="#003366"),
    ),
    xaxis=dict(tickfont=dict(size=20)),
    yaxis=dict(tickfont=dict(size=20), title="Number of 98 TFs included"),
    font=dict(size=20),
)
fig.show()
save_fig(name="exps_98_tfs", fig=fig, base_path=figures_path, width=1800, height=800, margin=0.06)

In [50]:
stats_quantile = exp_results_qth.groupby("Quantile")["Degree_th"].agg(["mean", "std"]).reset_index()
fig = px.line(stats_quantile, x="Quantile", y="mean", error_y="std", title="Degree ranges")
fig = fig.update_layout(title="", xaxis=dict(tickfont=dict(size=24)), yaxis=dict(tickfont=dict(size=24), title="Mean degreee"), font=dict(size=24), height=700)
fig.show()
save_fig(name="exp_degree_th", fig=fig, base_path=figures_path, width=1000, height=500, margin=0.02)

## Control experiments

### When are all the 98 TFs genes included?

With varying the quantile threshold are the 98 TFs subset selected?

From the experiments below it can be seen that q_th needs to be set to 0.4


In [15]:
q_ths = list(range(35, 100, 5))
q_ths.sort()
stats = [(0, len(t_sel_tfs))]  # default values
for q_th in q_ths:
    q_th = q_th / 100

    common_ctrls = set()
    for key, value in exp_ctrls.items():
        label = key.split("t")[-1]
        matched_ctrl, fig_ctrl, meta_info, global_stats = calculate_degree_thresholds(exp_set=value, sel_tfs=t_sel_tfs, label=label, q_th=q_th, verbose=False)

        common_ctrls.update(matched_ctrl)

    stats.append((q_th, len(common_ctrls)))
    print(f"### Matched (all ctrls & 98 TFs) {len(common_ctrls)}")

stats = pd.DataFrame(stats, columns=["Quantile", "Matched"])
stats.sort_values(by="Quantile", ascending=True, inplace=True)
stats["Quantile"] = stats["Quantile"].astype(str)

px.bar(stats, x="Quantile", y="Matched", text_auto=True, title="Matched across all controls")

### Matched (all ctrls & 98 TFs) 125
### Matched (all ctrls & 98 TFs) 124
### Matched (all ctrls & 98 TFs) 124
### Matched (all ctrls & 98 TFs) 124
### Matched (all ctrls & 98 TFs) 124
### Matched (all ctrls & 98 TFs) 121
### Matched (all ctrls & 98 TFs) 118
### Matched (all ctrls & 98 TFs) 113
### Matched (all ctrls & 98 TFs) 107
### Matched (all ctrls & 98 TFs) 97
### Matched (all ctrls & 98 TFs) 81
### Matched (all ctrls & 98 TFs) 65
### Matched (all ctrls & 98 TFs) 38


### Matched controls across TFs

The limitation with the above graph is that in looks at all the control which gives an unrealistic representation.

In the following section we look only at the ones

In [16]:
all_q_ths = pd.DataFrame()
q_ths = list(range(40, 100, 5))
q_ths.sort()
stats = []
for q_th in q_ths:
    q_th = q_th / 100

    common_ctrls = set()
    all_ctrls = pd.DataFrame()
    for key, value in exp_ctrls.items():
        label = key.split("h")[-1]
        matched_ctrl, fig_ctrl, meta_info, global_stats = calculate_degree_thresholds(exp_set=value, sel_tfs=t_sel_tfs, label=label, q_th=q_th, verbose=False)

        common_ctrls.update(matched_ctrl)
        meta_info["Quantile"] = q_th
        all_ctrls = pd.concat([all_ctrls, meta_info], axis=0)

    all_q_ths = pd.concat([all_ctrls, all_q_ths], axis=0)

# Computer TF and Quantile
stats = []
for tf in all_q_ths["TF"].unique():
    sel_dmy = all_q_ths.loc[all_q_ths["TF"] == tf]

    for q_th in sel_dmy["Quantile"].unique():
        sel = sel_dmy[sel_dmy["Quantile"] == q_th]
        stats.append(
            (
                tf,
                q_th,
                sel["matched"].mean(),
                sel["matched"].std(),
            )
        )

In [17]:
stats = pd.DataFrame(stats, columns=["TF", "q_th", "Mean", "Std"])

fig = px.line(stats, x="TF", y="Mean", error_y="Std", color="q_th", color_discrete_sequence=px.colors.qualitative.G10 + px.colors.qualitative.D3, height=700)
fig = fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        xanchor="center",
        x=0.5,
        y=1.0,
        bgcolor="rgba(0,0,0,0)",
        font=dict(size=20, color="#003366"),
    ),
    xaxis=dict(tickfont=dict(size=20)),
    yaxis=dict(tickfont=dict(size=20), title="Mean of 98 TFs included"),
    font=dict(size=20),
)
save_fig(name="ctrls_98_tfs", fig=fig, base_path=figures_path, width=1800, height=800, margin=0.06)

In [18]:
stats_quantile = all_q_ths.groupby("Quantile")["Degree_th"].agg(["mean", "std"]).reset_index()
fig = px.line(stats_quantile, x="Quantile", y="mean", error_y="std", title="Degree ranges")
fig = fig.update_layout(title="", xaxis=dict(tickfont=dict(size=24)), yaxis=dict(tickfont=dict(size=24), title="Mean degreee"), font=dict(size=24), height=700)
save_fig(name="ctrl_degree_th", fig=fig, base_path=figures_path, width=1000, height=500, margin=0.02)

# Check for specific TF

We found the 98 TFs by varying the number of TFs and then doing the interesection. However, can we just use one value for TF?

In [19]:
q_th = 0.98
matched_exp, fig_exp, exps_th, global_stats = calculate_degree_thresholds(exp_set=exp_set, sel_tfs=t_sel_tfs, label="Exp", q_th=q_th, verbose=False)
fig_exp.show()

# Reproduce inclusion of the 98 TFs

First gather all the genes from each experiment

Then do the same thing for controls

Look at how many of the initial are included in the selected

In [23]:
def extract_genes(exp_set: ExperimentSet, tf_list: list, exp_type: str, q_th=0.5):
    stats_exp = []
    info_exp = {}
    for key, nodes_df in exp_set.items():
        tf = NetworkOutput.extract_tf_number(key)
    
        median = nodes_df['degree_t'].quantile(q=q_th)
        all_tf = len(set(nodes_df.index) & set(tf_list))

        genes_sel = nodes_df[nodes_df['degree_t'] >= median][['degree_t', 'TF', 'Type']]
        info_exp[tf] = genes_sel

        tfs_included = len(set(genes_sel.index) & set(tf_list))
        stats_exp.append((
            tf,
            genes_sel.shape[0],
            tfs_included,
            round(tfs_included / all_tf * 100, 4),
            exp_type
        ))

    stats_exp = pd.DataFrame(stats_exp, columns=['TF', 'Num genes', 'Num TF', "Prct TF", 'Type'])
    stats_exp['TF'] = stats_exp['TF'].astype(int)
    stats_exp.sort_values(by='TF', ascending=True, inplace=True)
    return stats_exp


#### Run exp
stats_exp = extract_genes(exp_set=exp_set, tf_list=tf_list, exp_type='Exp', q_th=0.5)

#### Run ctrls
all_ctrls = pd.DataFrame()
for key, ctrl_exp in exp_ctrls.items():
    ctrl_stats = extract_genes(exp_set=ctrl_exp, tf_list=tf_list, exp_type=key, q_th=0.5)
    all_ctrls = pd.concat([all_ctrls, ctrl_stats], axis=0)

all_ctrls.sort_values(by='TF', ascending=True, inplace=True)

In [21]:
all_tf_chgs_stats = []
for tf in all_ctrls['TF'].unique():
    sel_df = all_ctrls[all_ctrls['TF'] == tf]
    all_tf_chgs_stats.append([
        tf,
        sel_df['Prct TF'].median(axis=0),
        sel_df['Prct TF'].mean(axis=0),
        sel_df['Prct TF'].std(axis=0),
        sel_df['Num TF'].median(axis=0),
        sel_df['Num TF'].mean(axis=0),
        sel_df['Num TF'].std(axis=0),
    ])    


tf_chgs_stats_df = pd.DataFrame(
    all_tf_chgs_stats,
    columns=["TF", "Prct_median", "Prct_mean", "Prct_std", "#TF_median", "#TF_mean", "#TF_std"],
)
tf_chgs_stats_df["Type"] = "Control"


## Experiment
used_tf = set(sel_exp_42.tpm_df.index) & set(tf_list)
stats_exp["Used_prct"] = stats_exp["Num TF"] / len(used_tf) * 100
stats_exp["Prct_median"] = stats_exp["Used_prct"]
stats_exp["Prct_mean"] = stats_exp["Used_prct"]
stats_exp["Prct_std"] = 0
stats_exp["Type"] = "Experiment"

## Combine the two
comb_chgs = pd.concat([tf_chgs_stats_df, stats_exp], axis=0)
comb_chgs.sort_values(by='TF', ascending=True, inplace=True)
comb_chgs['TF'] = comb_chgs["TF"].astype(str)


In [22]:
color_map = {"Experiment": px.colors.qualitative.G10[0], "Control": px.colors.qualitative.G10[1]}

fig = px.line(
    comb_chgs,
    x="TF",
    y="Prct_mean",
    error_y="Prct_std",
    color="Type",
    color_discrete_map=color_map,
    markers=True,
    title="% TF used in calculating the MEV",
    height=600,
)
fig.update_layout(
    legend=dict(
        orientation="h",
        # title="Type",
        yanchor="bottom",
        y=0.18,
        xanchor="center",
        x=0.5,
        bgcolor="rgba(0,0,0,0)",
        font=dict(size=16),
    ),
    title="",
    xaxis=dict(
        tickfont=dict(size=16),
        title_font=dict(
            size=16,
        ),
        title="Minimum degree for selected genes",
    ),
    yaxis=dict(
        tickfont=dict(size=16),
        title_font=dict(
            size=16,
        ),
        title="% of genes included",
        # range=[0, 25],
    ),
    font=dict(size=16),
)
# fig = fig.update_yaxes(title_text="% of genes included", range=[0, 105], tickfont=dict(size=14), title_font=dict(size=16))
fig.update_traces(line_width=5, marker_size=12, error_y=dict(width=5))
save_fig(name="ctrls_min_dig_mev", fig=fig, base_path=figures_path, width=1200, height=600)
fig.show()

# Export TFs for Morpheus

The goal is to re-create the Morpheus heatmap. How many genes can we afford to loose?

In [36]:
q_th = 50/100

common_ctrls = set()
all_ctrls = pd.DataFrame()
for key, value in exp_ctrls.items():
    label = key.split("h")[-1]
    matched_ctrl, fig_ctrl, meta_info, global_stats = calculate_degree_thresholds(exp_set=value, sel_tfs=sel_tfs, label=label, q_th=q_th, verbose=False)


print("How many are missed using the degree th of {}? {}".format(q_th,len(set(sel_tfs.index) - set(matched_ctrl))))


NameError: name 'sel_tfs' is not defined

# Without matching the 98 TFs

Find the genes common without the restrictions of the 98 TFs

In [152]:
results_qth = {}
q_th = 0.9
matched_exp, fig_exp, meta_info, global_stats = calculate_degree_thresholds(exp_set=exp_set, sel_tfs=pd.DataFrame(), label="Exp", q_th=q_th, verbose=False)

common_ctrls = set()
all_ctrls = pd.DataFrame()
for key, value in exp_ctrls.items():
    label = key.split("h")[-1]
    matched_ctrl, fig_ctrl, meta_info, global_stats = calculate_degree_thresholds(exp_set=value, sel_tfs=t_sel_tfs, label=label, q_th=q_th, verbose=False)

    common_ctrls.update(matched_ctrl)
    meta_info["Quantile"] = q_th
    all_ctrls = pd.concat([all_ctrls, meta_info], axis=0)

print(f"### Q {q_th}. Matched genes across controls {len(common_ctrls)}")

matched_overall = set(matched_exp) & set(common_ctrls)
results_qth[q_th] = list(matched_overall)
print(f"### Overall matched (ctrls & exps) {len(matched_overall)}")

### Q 0.9. Matched genes across controls 65
### Overall matched (ctrls & exps) 65


In [153]:
# tpm_sel = tum_tpms_v4.loc[list(matched_overall)]
len(set(tf_list) & set(matched_overall))

65

# Clustering the 98 TF expression

So far we have found that the 98 TFs can be selected by putting a threshold on the degree, but we don't know if we can only use the gene expression.

In [25]:
outliers_log2 = [
    "TCGA-C4-A0EZ",
    "TCGA-DK-AA6W",
    "TCGA-G2-A2EL",
    "TCGA-BL-A3JM",
    "TCGA-XF-A9T2",
    "TCGA-XF-AAMH",
    "TCGA-XF-A9ST",
    "TCGA-GC-A4ZW",
    "TCGA-HQ-A2OF",
    "TCGA-DK-AA6T",
    "TCGA-BT-A2LA",
    "TCGA-XF-AAN7",
    "TCGA-FJ-A871",
    "TCGA-CF-A3MF",
]

norm_outliers = [
    "TCGA-2F-A9KW",
    "TCGA-XF-A9ST",
    "TCGA-BL-A3JM",
    "TCGA-XF-A9T2",
    "TCGA-XF-AAMH",
    "TCGA-DK-AA6T",
    "TCGA-BT-A2LA",
    "TCGA-XF-AAN7",
    "TCGA-FJ-A871",
    "TCGA-C4-A0EZ",
    "TCGA-DK-AA6W",
    "TCGA-G2-A2EL",
]

cmn_outliars = set(outliers_log2) & set(norm_outliers)

print(f"### Num outliers for standard log2 {len(outliers_log2)}.\n --> {outliers_log2}")
print(f"### Num outliers for norm log2 {len(norm_outliers)}.\n --> {norm_outliers}")
print(f"### Common outliers *{len(cmn_outliars)}*.\n --> {cmn_outliars}")

### Num outliers for standard log2 14.
 --> ['TCGA-C4-A0EZ', 'TCGA-DK-AA6W', 'TCGA-G2-A2EL', 'TCGA-BL-A3JM', 'TCGA-XF-A9T2', 'TCGA-XF-AAMH', 'TCGA-XF-A9ST', 'TCGA-GC-A4ZW', 'TCGA-HQ-A2OF', 'TCGA-DK-AA6T', 'TCGA-BT-A2LA', 'TCGA-XF-AAN7', 'TCGA-FJ-A871', 'TCGA-CF-A3MF']
### Num outliers for norm log2 12.
 --> ['TCGA-2F-A9KW', 'TCGA-XF-A9ST', 'TCGA-BL-A3JM', 'TCGA-XF-A9T2', 'TCGA-XF-AAMH', 'TCGA-DK-AA6T', 'TCGA-BT-A2LA', 'TCGA-XF-AAN7', 'TCGA-FJ-A871', 'TCGA-C4-A0EZ', 'TCGA-DK-AA6W', 'TCGA-G2-A2EL']
### Common outliers *11*.
 --> {'TCGA-BT-A2LA', 'TCGA-XF-AAN7', 'TCGA-XF-AAMH', 'TCGA-XF-A9T2', 'TCGA-DK-AA6W', 'TCGA-BL-A3JM', 'TCGA-XF-A9ST', 'TCGA-FJ-A871', 'TCGA-DK-AA6T', 'TCGA-C4-A0EZ', 'TCGA-G2-A2EL'}


In [26]:
dmy_df = tum_tpms_v4.loc[sel_tfs.index].copy(deep=True)
dmy_df = np.log2(dmy_df + 1)

dmy_df = dmy_df[list(set(dmy_df.columns) - set(cmn_outliars))]

sel_metadata = [
    "KMeans_labels_6",
    "consensus",
    "TCGA408_classifier",
    "Lund2017.subtype",
    "ESTIMATE_score",
    "Immune_score",
    "Stromal_score",
    "tumor_stage",
    "Net_I",
]
dmy_df = pd.concat([vu_output[sel_metadata].T, dmy_df], axis=0).dropna(axis=1)


dmy_df.to_csv(f"{figures_path}/log2_sel_tfs_exp_v1.1.tsv", sep="\t")